In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

import datetime

In [2]:
DAILY_C = pd.read_csv("markets_historical_kc1_com.csv", parse_dates=True, dayfirst=True)
DAILY_C['Date'] = DAILY_C.apply(lambda x: datetime.datetime.strptime(x['Date'], '%d/%m/%Y'), axis=1)
DAILY_C = DAILY_C[['Date', 'Close']]

In [3]:
def country_inflator (country, date_threshold):
    
    filename = "historical_country_" + country + "_indicator_Inflation_Rate.csv"
    df = pd.read_csv(filename, parse_dates=True)
    df['Date'] = df.apply(lambda x: datetime.datetime.strptime(x['DateTime'], '%Y-%m-%dT%H:%M:%S'), axis=1)
    df = df[df['Date']>=date_threshold]
    df.reset_index(inplace=True)

    df['Inflator'] = 1.
    for i, v in df.iterrows():
        if i >= 12:
            df['Inflator'][i] = df['Inflator'][i-12] * (1. + df['Value'][i]/100)
    base = float(df['Inflator'][-1:])
    for i, v in df.iterrows():
        df['Inflator'][i] /= base

    df = df[['Date', 'Country', 'Inflator']]
    df.dropna(inplace=True)
    return df

In [4]:
def exchange_rate (country, date_threshold):
    
    if country == "United_States":
        return pd.DataFrame({'Date': list(DAILY_C['Date']), 'FX': 1.})
    
    filename = "historical_country_" + country.lower() + "_indicator_currency.csv"
    df = pd.read_csv(filename, parse_dates=True)
    df['Date'] = df.apply(lambda x: datetime.datetime.strptime(x['DateTime'], '%Y-%m-%dT%H:%M:%S'), axis=1)
    df = df[['Date', 'Value']]
    df.columns = ['Date', 'FX']
    return df

In [5]:
def price_adjustor (c_prices, country, date_threshold):
    
    c_prices = c_prices[c_prices['Date']>=date_threshold]
    today_price = float(c_prices['Close'][-1:])
    
    inflation_data = country_inflator(country, START_DATE)
    fx_data = exchange_rate(country, START_DATE)
    today_fx = float(fx_data['FX'][-30:].mean())
    
    df = pd.merge_asof(c_prices, inflation_data, on='Date')
    df = pd.merge_asof(df, fx_data, on='Date')
    df.dropna(inplace=True)
    
    df["Adjusted C Price"] = df.apply(lambda x:
                               (x['Close'] * x['FX'] / x['Inflator']) /
                               (today_price * today_fx) * today_price
                              , axis=1)
    
    df.reset_index(inplace=True)
    df = df[["Date", "FX", "Inflator", "Close", "Adjusted C Price"]]
    df.rename(columns={"Close": "Nominal C Price"}, inplace=True)
    return df

In [6]:
def get_summary_indicators (country, df):
    
    cum_inflation = float(df['Inflator'][-1:] / df['Inflator'][0])
    cum_devaluation = float(df['FX'][-30:].mean() / df['FX'][0].mean())
    factor = cum_inflation / cum_devaluation
    
    return {
        'Country': country,
        'Cumulative Inflation': cum_inflation, 
        'Cumulative Devaluation': cum_devaluation, 
        'Adj. Factor': factor
    }

In [7]:
COUNTRIES = ["Brazil", "Colombia", "Indonesia", "Vietnam", "Guatemala", "Ethiopia"]
START_DATE = datetime.datetime(2001,8,28)

summary_table = pd.DataFrame(columns=('Country', 'Cumulative Inflation', 'Cumulative Devaluation', 'Adj. Factor'))

all_countries = price_adjustor(DAILY_C, "United_States", START_DATE)
for country in COUNTRIES:
    temp = price_adjustor(DAILY_C, country, START_DATE)
    summary_row = get_summary_indicators(country, temp)
    summary_table = summary_table.append(summary_row, ignore_index=True)

    temp = temp[['Date', 'Adjusted C Price']]
    temp.columns = ['Date', country]
    all_countries = all_countries.merge(temp, on='Date', how='left')

all_countries.fillna(value='', inplace=True)

In [8]:
summary_table.to_csv("summary_analysis.csv")
summary_table

,Country,Cumulative Inflation,Cumulative Devaluation,Adj. Factor
0,Brazil,2.680440,1.516248,1.767811
1,Colombia,2.028681,1.276301,1.589500
2,Indonesia,2.686708,1.638117,1.640120
3,Vietnam,3.204364,1.551429,2.065427
4,Guatemala,2.282676,0.947502,2.409151
5,Ethiopia,8.516746,3.262073,2.610839


In [9]:
all_countries.head(1)

,Date,FX,Inflator,Nominal C Price,Adjusted C Price,Brazil,Colombia,Indonesia,Vietnam,Guatemala,Ethiopia
0,2001-08-31,1.0,0.715878,50.5,70.542791,88.46652,79.846525,82.687461,109.514834,121.575517,131.775545


In [10]:
ET = price_adjustor(DAILY_C, "Ethiopia", START_DATE)

In [11]:
ET.head()

,Date,FX,Inflator,Nominal C Price,Adjusted C Price
0,2001-08-31,8.4,0.117416,50.50,131.775545
1,2001-09-04,8.4,0.117416,51.05,133.210724
2,2001-09-05,8.4,0.117416,48.50,126.556712
3,2001-09-06,8.4,0.117416,52.15,136.081083
4,2001-09-07,8.4,0.117416,51.80,135.167787


In [12]:
import plotly
plotly.tools.set_credentials_file(username='numberscoffee', api_key='jOO5qNjOFSNJUqXOGNDA')

In [13]:
import plotly.plotly as py

df = DAILY_C[DAILY_C['Date']>="2000-01-01"]
df = df[df['Date']<"2004-12-31"]

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Close'], 'mode': 'line'}
    ],
    'layout': {
        'title': 'Arabica coffee prices during the "coffee crisis"',
        'yaxis': {'title': "US cents per pound"}
    }
}

py.image.save_as(fig, filename='coffee-crisis.png')
py.iplot(fig, filename='coffee-crisis')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~numberscoffee/0 or inside your plot.ly account where it is named 'coffee-crisis'


In [14]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Adjusted C Price'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
        
    ],
    'layout': {
        'title': 'Arabica coffee prices, nominal and inflation-adjusted',
        'yaxis': {'title': "US cents per pound"}

    }
}

py.image.save_as(fig, filename='inflation-adjusted.png')
py.iplot(fig, filename='inflation-adjusted')

In [15]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Brazil'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
    ],
    'layout': {
        'title': 'Brazil: Arabica coffee prices, nominal and adjusted',
        'yaxis': {'title': "US cents per pound"}
    }
}
    
py.image.save_as(fig, filename='brazil.png')
py.iplot(fig, filename='brazil')

In [16]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Colombia'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
    ],
    'layout': {
        'title': 'Colombia: Arabica coffee prices, nominal and adjusted',
        'yaxis': {'title': "US cents per pound"}
    }
}

py.image.save_as(fig, filename='colombia.png')
py.iplot(fig, filename='colombia')

In [17]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Guatemala'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
    ],
    'layout': {
        'title': 'Guatemala: Arabica coffee prices, nominal and adjusted',
        'yaxis': {'title': "US cents per pound"}
    }
}

py.image.save_as(fig, filename='guatemala.png')
py.iplot(fig, filename='guatemala')

In [18]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Vietnam'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
    ],
    'layout': {
        'title': 'Vietnam: Arabica coffee prices, nominal and adjusted',
        'yaxis': {'title': "US cents per pound"}
    }
}
    
py.image.save_as(fig, filename='vietnam.png')    
py.iplot(fig, filename='vietnam')

In [19]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Indonesia'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
    ],
    'layout': {
        'title': 'Indonesia: Arabica coffee prices, nominal and adjusted',        
        'yaxis': {'title': "US cents per pound"}
    }
}
    
py.image.save_as(fig, filename='indo.png')
py.iplot(fig, filename='indo')

In [20]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Ethiopia'], 'mode': 'line', 'name': 'Inflation-adjusted price'},
    ],
    'layout': {
        'title': 'Ethiopia: Arabica coffee prices, nominal and adjusted',        
        'yaxis': {'title': "US cents per pound"}
    }
}
    
py.image.save_as(fig, filename='ethiopia.png')
py.iplot(fig, filename='ethiopia')

In [21]:
import plotly.plotly as py

df = all_countries

fig = {
    'data': [
        {'x': df['Date'], 'y': df['Nominal C Price'], 'mode': 'line', 'name': 'Nominal price'},
        {'x': df['Date'], 'y': df['Brazil'], 'mode': 'line', 'name': 'Brazil'},
        {'x': df['Date'], 'y': df['Colombia'], 'mode': 'line', 'name': 'Colombia'},
        {'x': df['Date'], 'y': df['Vietnam'], 'mode': 'line', 'name': 'Vietnam'},
        {'x': df['Date'], 'y': df['Indonesia'], 'mode': 'line', 'name': 'Indonesia'},
        {'x': df['Date'], 'y': df['Guatemala'], 'mode': 'line', 'name': 'Guatemala'},
        {'x': df['Date'], 'y': df['Ethiopia'], 'mode': 'line', 'name': 'Ethiopia'},
    ],
    'layout': {
        'title': 'Selected countries: Arabica coffee prices, nominal and adjusted',
        'yaxis': {'title': "US cents per pound"}
    }
}
    

py.image.save_as(fig, filename='country-adjusted.png')
py.iplot(fig, filename='country-adjusted')

In [22]:
all_countries.head()

,Date,FX,Inflator,Nominal C Price,Adjusted C Price,Brazil,Colombia,Indonesia,Vietnam,Guatemala,Ethiopia
0,2001-08-31,1.0,0.715878,50.50,70.542791,88.466520,79.846525,82.687461,109.514834,121.575517,131.775545
1,2001-09-04,1.0,0.715878,51.05,71.311078,89.780723,80.921262,84.785769,110.722337,122.432899,133.210724
2,2001-09-05,1.0,0.715878,48.50,67.749017,85.296083,77.117343,81.133035,105.198657,116.465048,126.556712
3,2001-09-06,1.0,0.715878,52.15,72.847654,92.790061,83.227272,86.709034,113.138297,125.388865,136.081083
4,2001-09-07,1.0,0.715878,51.80,72.358744,91.811451,82.542395,87.323304,112.393963,124.705183,135.167787


In [23]:
vietnam = price_adjustor(DAILY_C, "Vietnam", START_DATE)

In [24]:
df = vietnam

In [25]:
cum_inflation = float(df['Inflator'][-1:] / df['Inflator'][0])
cum_devaluation = float(df['FX'][-30:].mean() / df['FX'][0:30].mean())
factor =cum_inflation / cum_devaluation

temp = pd.DataFrame(columns=('Country', 'Cumulative Inflation', 'Cumulative Devaluation', 'Adj. Factor'))
temp.append({
    'Country': 'Vietnam', 
    'Cumulative Inflation': cum_inflation, 
    'Cumulative Devaluation': cum_devaluation, 
    'Adj. Factor': factor
    }, ignore_index=True)

,Country,Cumulative Inflation,Cumulative Devaluation,Adj. Factor
0,Vietnam,3.204364,1.550029,2.067293


In [26]:
50.5 *  (cum_inflation / cum_devaluation)

104.39830709284848

In [27]:
vietnam.head()

,Date,FX,Inflator,Nominal C Price,Adjusted C Price
0,2001-08-31,14994.0,0.297011,50.50,109.514834
1,2001-09-04,14996.0,0.297011,51.05,110.722337
2,2001-09-05,14997.0,0.297011,48.50,105.198657
3,2001-09-06,15000.0,0.297011,52.15,113.138297
4,2001-09-07,15002.0,0.297011,51.80,112.393963


In [28]:
vietnam.tail()

,Date,FX,Inflator,Nominal C Price,Adjusted C Price
4245,2018-08-23,23281.0,0.951732,97.20,102.138656
4246,2018-08-24,23300.0,0.951732,100.55,105.745097
4247,2018-08-27,23293.0,0.951732,101.85,107.080084
4248,2018-08-28,23295.0,0.951732,99.25,104.355532
4249,2018-08-29,23301.0,0.951732,98.65,103.751383


In [29]:
# inflation
(.951732 / .297011) ** (1/(2018-2001)) -1

0.07090152623394408

In [30]:
# fx
(23.3 / 15.0) ** (1/(2018-2001)) -1

0.026244546952585823